# README
In this notebook, I will be looking at drought indicators at the watershed level in Baja Calidornia 
Sur. The inidcators evaluated are precipitation, land surface temperature, and ground water deviation
from basline (using GRACE data, assumes a baseline of 2004-2009). The time period evaluated for 
these datasets is 2013 - 2023.

For the purpose of this analysis, I will look at these indicators on an annual basis (taking a mean across months). Future work may entail digging into the variance within each year for these indicators. Future work may also entail testing as to whether or not the annual trends are statistically significant.

# Imports

In [1]:
import ee

from gee_water.utils import (
    annual_agg_ic,
    reduce_to_basin_means_annual,
    get_annual_pdf,
    get_basin_geodataframe,
    get_annual_data,
    get_annual_precip_data
)
from gee_water.analysis import get_slope
from gee_water.visualization_utils import create_choropleth_map

from functools import reduce

import pandas as pd


In [2]:
ee.Initialize()

# Global Variables

In [3]:
# For this particular blog, I am analyzing multiple datasets from a handful of different 
# satellites. The year 2013 is far enough back historically to capture form trends, but still 
# allows for higih quality data to be gathred for all my metrics of interest.
start_date = '2013-01-01'
end_date   = '2023-12-31'

# Define start and end year from your date strings
start_year = int(start_date[:4])
end_year   = int(end_date[:4])

In [4]:
# get watershed basin boundaries at desired level of granularity (12 is highest granularity, 1 is the most coarse)
BASINS_LEVEL9_ID = 'WWF/HydroSHEDS/v1/Basins/hybas_9'
BASINS_LEVEL8_ID = 'WWF/HydroSHEDS/v1/Basins/hybas_8'
BASINS_LEVEL7_ID = 'WWF/HydroSHEDS/v1/Basins/hybas_7' 

# geographical regional boundary dataset (country, state, etc.)
BOUNDARIES_ID = 'FAO/GAUL/2015/level1' 

PRECIP_GEE_ID = 'UCSB-CHG/CHIRPS/DAILY'
GW_GEE_ID = 'NASA/GRACE/MASS_GRIDS_V03/MASCON_CRI'
LST_GEE_ID = 'MODIS/061/MOD11A1'

In [5]:
baja_sur = ee.FeatureCollection(BOUNDARIES_ID).filter(
    'ADM0_NAME == "Mexico" && '
    'ADM1_NAME == "Baja California Sur"'
)

# want to ensure that no ocean water is contained within the geometry, as this may lead to 
# innacurate analysis of the GRACE dataset for ground water anomalies
# define inland mile buffer
mile_buffer = 1.5

# apply a negative buffer to each feature in the collection
baja_sur = baja_sur.map(lambda f: f.buffer(-mile_buffer* 1609.34))


In [6]:
EVALUATION_BASIN_ID = BASINS_LEVEL8_ID

In [7]:
baja_basins_gdf = get_basin_geodataframe(EVALUATION_BASIN_ID, bounding_geom=baja_sur)

# Precipitation

In [8]:
precip_df = get_annual_precip_data(gee_id=PRECIP_GEE_ID, basin_level_id=EVALUATION_BASIN_ID, start_date=start_date, end_date=end_date, bounding_geom=baja_sur)

## Evaluate Precip Annuals Trends

In [9]:
precip_slope_df = (
    precip_df.sort_values(['HYBAS_ID', 'year'])
    .groupby('HYBAS_ID', as_index=False)
    .apply(lambda group: get_slope(group, group_col_name='year', analysis_col_name='annual_precip_mm'))  
    .rename(columns={None: 'slope'})
)

/var/folders/k2/qdsrvv_d7ll8v_183kc33pp80000gn/T/ipykernel_34572/3377553992.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: get_slope(group, group_col_name='year', analysis_col_name='annual_precip_mm'))


In [10]:
# merge basin gdf with trend data for plotting
baja_basins_gdf_with_precip = baja_basins_gdf.merge(precip_slope_df, on='HYBAS_ID').rename(columns={'slope': 'average_chirps_precip_mm_per_year'})

In [11]:
create_choropleth_map(
    gdf=baja_basins_gdf_with_precip,
    color_by_value='average_chirps_precip_mm_per_year',
    color_continuous_scale=[
            (0.00, "red"),   # 0% of the range -> blue
            (0.50, "white"),  # 50% of the range -> white (mid)
            (1.00, "blue")     # 100% of the range -> red
        ],
    color_continuous_midpoint=0,
    legend_title="Average Annual<br> Change in<br> Precipitation<br> 2013 - 2023 (mm)<br>",
    legend_font_size=12,
    html_export_location='./../static/html/basin_precip_map.html'
)

# Ground Water Anomolies

In [12]:
total_water_pdf = get_annual_data(
    gee_id=GW_GEE_ID, 
    basin_level_id=EVALUATION_BASIN_ID, 
    start_date=start_date, 
    end_date=end_date, 
    bounding_geom=baja_sur, 
    annual_agg_type='mean', 
    output_col_name='lwc_mean_thickness_cm', # Equivalent liquid water thickness in centimeters, representing water storage/height anomalies
    band_name='lwe_thickness'
)


## Evaluate Near Surface Water Anomolies (Soil Moistures and SWE)

In [13]:
SM_BASELINE_START = '2004-01-01'
SM_BASELINE_END = '2009-12-31'

SM_GEE_ID = 'NASA/GLDAS/V021/NOAH/G025/T3H'

NEAR_SURFACE_MOISTURE_BANDS  =  [
    # 'RootMoist_inst',
    'SoilMoi0_10cm_inst',
    'SoilMoi10_40cm_inst',
    'SoilMoi40_100cm_inst',
    'SoilMoi100_200cm_inst',
    'SWE_inst'
]

In [14]:
def get_baseline_soil_moisture_df(band_name):
    return get_annual_data(
        GEE_ID=SM_GEE_ID, 
        basin_level_id=EVALUATION_BASIN_ID,
        start_date=SM_BASELINE_START,
        end_date=SM_BASELINE_END,
        bounding_geom=baja_sur,
        annual_agg_type='mean',  # we want annual means
        output_col_name=band_name,  
        band_name=band_name
    )


def get_analysis_soil_moisture_df(band_name):
    return get_annual_data(
        GEE_ID=SM_GEE_ID, 
        basin_level_id=EVALUATION_BASIN_ID,
        start_date=start_date,
        end_date=end_date,
        bounding_geom=baja_sur,
        annual_agg_type='mean', 
        output_col_name=band_name,  
        band_name=band_name
    )

In [15]:
# get baseline dataframes for soil moisture bands
baseline_sm_dfs = [get_baseline_soil_moisture_df(b) for b in NEAR_SURFACE_MOISTURE_BANDS]


TypeError: get_annual_data() got an unexpected keyword argument 'GEE_ID'

In [ ]:
# merge
baseline_soil = reduce(
    lambda left, right: pd.merge(left, right, on=['HYBAS_ID', 'year'], how='outer'),
    baseline_sm_dfs
)

# Create a "total_sm" column (kg/m²)
baseline_soil['total_sm'] = (
    baseline_soil['SoilMoi0_10cm_inst'] +
    baseline_soil['SoilMoi10_40cm_inst'] +
    baseline_soil['SoilMoi40_100cm_inst'] +
    baseline_soil['SoilMoi100_200cm_inst'] + 
    baseline_soil['SWE_inst']
)
# baseline_soil['total_sm'] = baseline_soil['RootMoist_inst'] 


# compute the average across the baseline years (2004–2009) for each basin
# group by HYBAS_ID, then take the mean of 'total_sm'.
baseline_avg = (
    baseline_soil
    .groupby('HYBAS_ID', dropna=False)['total_sm']
    .mean()
    .reset_index()
    .rename(columns={'total_sm': 'baseline_mean_kg_m2'})
)

In [ ]:
analysis_dfs = [get_analysis_soil_moisture_df(b) for b in NEAR_SURFACE_MOISTURE_BANDS]

In [ ]:
analysis_soil = reduce(
    lambda left, right: pd.merge(left, right, on=['HYBAS_ID', 'year'], how='outer'),
    analysis_dfs
)

analysis_soil['total_sm'] = (
    analysis_soil['SoilMoi0_10cm_inst'] +
    analysis_soil['SoilMoi10_40cm_inst'] +
    analysis_soil['SoilMoi40_100cm_inst'] +
    analysis_soil['SoilMoi100_200cm_inst'] + 
    analysis_soil['SWE_inst']
)

In [ ]:
# compute soil moisture annual anomaly
analysis_soil = pd.merge(
    analysis_soil,
    baseline_avg[['HYBAS_ID','baseline_mean_kg_m2']],
    on='HYBAS_ID',
    how='left'
)

analysis_soil['sm_anomaly_kg_m2'] = (
    analysis_soil['total_sm'] 
  - analysis_soil['baseline_mean_kg_m2'] 
)

# convert from kg/m² (mm) -> cm
analysis_soil['sm_anomaly_cm'] = analysis_soil['sm_anomaly_kg_m2'] * 0.1

# keep only columns we need:
analysis_soil_anomalies_pdf = analysis_soil[['HYBAS_ID','year','sm_anomaly_cm']]

In [ ]:
# merge soil moisutre with grace data
combined_water_anomaly_pdf = pd.merge(
    total_water_pdf, 
    analysis_soil_anomalies_pdf, 
    on=['HYBAS_ID','year'], 
    how='outer'
)

In [ ]:
combined_water_anomaly_pdf['gw_anomaly_cm'] = combined_water_anomaly_pdf['lwc_mean_thickness_cm'] - combined_water_anomaly_pdf['sm_anomaly_cm']


## Evaluate ground water anomaly trends

In [ ]:
gwa_slope_df = (
    combined_water_anomaly_pdf.sort_values(['HYBAS_ID', 'year'])
    .groupby('HYBAS_ID', as_index=False)
    .apply(lambda group: get_slope(group, group_col_name='year', analysis_col_name='gw_anomaly_cm'))  
    .rename(columns={None: 'slope'})
)

In [ ]:
baja_basins_gdf_with_gwa = baja_basins_gdf.merge(gwa_slope_df, on='HYBAS_ID').rename(columns={'slope': 'avg_annual_delta_gwa'})

In [ ]:
# plot
create_choropleth_map(
    gdf=baja_basins_gdf_with_gwa,
    color_by_value='avg_annual_delta_gwa',
    color_continuous_scale=[
        (0.00, "red"),   # 0% of the range -> blue
        (0.50, "white"),  # 50% of the range -> white (mid)
        (1.00, "blue")     # 100% of the range -> red
    ],
    color_continuous_midpoint=0,
    legend_title="Annual Ground Water<br> Anomly Trend (cm)",
    legend_font_size=12,
    html_export_location='./../static/html/basin_gwa_map.html'
)

## Plot Annual Precipitation and Ground Water Trends at higher basin granularity

# Land Surface Temperature 

In [ ]:
lst_ic = get_annual_data(
    GEE_ID=LST_GEE_ID, 
    basin_level_id=EVALUATION_BASIN_ID, 
    start_date=start_date, 
    end_date=end_date, 
    bounding_geom=baja_sur, 
    annual_agg_type='mean', 
    output_col_name='lst_mean_deg_c', # Equivalent liquid water thickness in centimeters, representing water storage/height anomalies
    band_name='LST_Day_1km'
)

## Evaluate Land Surface Temperature Trends

In [ ]:
lst_slope_df = (
    lst_ic.sort_values(['HYBAS_ID', 'year'])
    .groupby('HYBAS_ID', as_index=False)
    .apply(lambda group: get_slope(group, group_col_name='year', analysis_col_name='lst_mean_deg_c'))  
    .rename(columns={None: 'slope'})
)

In [ ]:
baja_basins_gdf_with_lst = baja_basins_gdf.merge(lst_slope_df, on='HYBAS_ID').rename(columns={'slope': 'avg_annual_delta_lst'})

In [ ]:
create_choropleth_map(
    gdf=baja_basins_gdf_with_lst,
    color_by_value='avg_annual_delta_lst',
    color_continuous_scale=[
        (0.00, "blue"),   # 0% of the range -> blue
        (0.50, "white"),  # 50% of the range -> white (mid)
        (1.00, "red")     # 100% of the range -> red
    ],
    color_continuous_midpoint=0,
    legend_title="Annual Land Surface <br> Temperature Anomly <br> Trend (deg C)",
    legend_font_size=12,
    html_export_location='./../static/html/basin_lst_map.html'
)